In [9]:
import recbole
recbole.__version__

'0.2.1'

In [10]:
import tqdm

In [11]:
from recbole.evaluator.metrics import *

In [12]:
import pandas as pd

df_pred = pd.read_csv('./preds_full.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.relevance.value_counts()

0.0    21307932
4.5       25887
5.0       12924
Name: relevance, dtype: int64

In [13]:
df_pred_new.head()

,user_id,item_id,score,relevance
0,41648,2019,0.610297,0.0
1,41648,7361,0.530178,0.0
2,41648,5618,0.468824,0.0
3,41648,1617,0.427014,0.0
4,41648,5952,0.415578,0.0


In [44]:
recommendations = {}
pos_len = []
pos_idx_ = []

for user in tqdm.tqdm(sorted(df_pred_new.user_id.unique())):
    current = (
        df_pred_new[df_pred_new.user_id == user].sort_values('score', ascending=False).relevance > 0
    ).astype(int).tolist()
    
    pos_idx_.append(current)
    pos_len.append(sum(current))

100%|██████████| 2733/2733 [01:19<00:00, 34.31it/s]


In [45]:
pos_len = np.array(pos_len)

In [46]:
pos_len

array([  1,  15,  15, ..., 172,   9,  82])

In [47]:
min_len = min(map(lambda x: len(x), pos_idx_))

In [48]:
pos_idx = np.array([x[:min_len] for x in pos_idx_])

In [49]:
pos_idx = np.array(pos_idx)

In [50]:
pos_len

array([  1,  15,  15, ..., 172,   9,  82])

In [51]:
pos_idx

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [52]:
min_len

6967

Just averaging all user scores:
https://github.com/RUCAIBox/RecBole/blob/master/recbole/evaluator/evaluators.py#L140

In [53]:
np.mean(precision_(pos_idx, pos_len)[:,19])

0.057519209659714604

In [54]:
np.mean(recall_(pos_idx, pos_len)[:,19])

0.09632131280310642

In [55]:
np.mean(ndcg_(pos_idx, pos_len)[:,19])

0.09326437896736814

In [56]:
np.mean(map_(pos_idx, pos_len)[:,19])

0.03927176078866248

In [57]:
np.mean(mrr_(pos_idx, pos_len)[:,19])

0.18632413351027097

In [58]:
np.mean(hit_(pos_idx, pos_len)[:,19])

0.47457006952067327

In [59]:
auc_(
    np.array((df_pred_new['relevance'] > 0).astype(int).tolist()),
    np.array(df_pred_new['score'].tolist())
)

0.686625474152309

In [60]:
pos_rank_sum = np.array(
    [sum(np.where(x)[0]) for x in pos_idx]
)

In [61]:
user_len_list = np.array([len(df_pred_new[df_pred_new.user_id == user]) for user in sorted(df_test.user_id.unique())])

In [62]:
gauc_(
    user_len_list=user_len_list, 
    pos_len_list=pos_len, 
    pos_rank_sum=pos_rank_sum
)

0.8581787135535387